In [1]:
import torch
import pandas as pd
import random
from epifit.network_generation import generate_random_network
from epifit.graphing_utils import *
from matplotlib import pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"

def Population(n:int, device= "cpu"):
  initial_population = { #a columnar DB somewhat reminds the Pandas DataFrame 
      "S":torch.ones(n).to(device),
      "E":torch.zeros(n).to(device),
      "I":torch.zeros(n).to(device),
      "R":torch.zeros(n).to(device),
  }
  initial_population["S"][0]=0 #first one infective
  initial_population["I"][0]=1 #first one infective
  return initial_population


In [74]:
n= 20
avgDegree= 8

graph=nx.dense_gnm_random_graph(n, n*avgDegree)
graph=nx.to_numpy_array(graph)
ISNet = torch.FloatTensor(graph).to(device)

population= Population(n, device=device)

_N= ISNet.size()[0]
_IS= ISNet
_EE= torch.eye(_N).to(device)
_II= torch.eye(_N).to(device)
_RR= torch.eye(_N).to(device)
_state= torch.stack((population["S"], population["E"], population["I"], population["R"]))
_L= torch.zeros_like(_state, device=device)
_ps= torch.tensor([[0, 0.4, 0, 0],
                   [0, 0, 0.2, 0],
                   [0, 0, 0, 0.2],
                   [0.2, 0, 0, 0]], device=device)


_L[0]= _state[0]*torch.matmul(_IS, _state[2])
_L[1]= torch.matmul(_state[1], _EE)
_L[2]= torch.matmul(_state[2], _II)
_L[3]= torch.matmul(_state[3], _RR)

_P= 1- torch.exp(torch.matmul(torch.log(1 - _ps.T), _L))
_stable_prob= 1- torch.sum(_P, 0)
_P= _P+ _state*_stable_prob
_U= torch.rand(_N).to(device)

for i in range(4):
    _U= _U- _P[i]
    _state[i]= _U<0
    _U= _U+_state[i]
    pass


tensor([0.5515, 0.1090, 0.2341, 0.8638, 0.9627, 0.8279, 0.1373, 0.4978, 0.9486,
        0.9051, 0.4584, 0.4649, 0.3967, 0.3520, 0.3091, 0.1010, 0.9783, 0.5234,
        0.6518, 0.5329], device='cuda:0')


In [75]:
print(_state)
print(_P)

tensor([[0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
         0., 1.],
        [0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]], device='cuda:0')
tensor([[0.0000, 0.6000, 0.6000, 1.0000, 0.6000, 0.6000, 0.6000, 0.6000, 0.6000,
         0.6000, 0.6000, 0.6000, 1.0000, 1.0000, 1.0000, 0.6000, 1.0000, 0.6000,
         0.6000, 0.6000],
        [0.0000, 0.4000, 0.4000, 0.0000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000,
         0.4000, 0.4000, 0.4000, 0.0000, 0.0000, 0.0000, 0.4000, 0.0000, 0.4000,
         0.4000, 0.4000],
        [0.8000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.2000, 0.000